# Utiliser les structures Rust depuis JS

---

> Dans la partie précédente, nous nous sommes contentés d'appeler des fonctions rust depuis JS, cependant cela pose certaines limitations comme la difficulté de stocker des données cote rust entre plusieurs appels successifs.

> On aimerait bien pouvoir exporter une structure avec ses méthodes depuis Rust et l'utiliser cote javascript comme s'il s'agissait d'un objet quelconque. C'est ce que nous allons voir dans ce chapitre.

## Le constructeur new

> La documentation de wasm-bindgen est suffisamment complète à ce sujet :

- The `wasm-bindgen` Guide chapitre **2.18.2.1** :

```
constructor

When attached to a Rust "constructor" it will make the generated JavaScript bindings callable as new Foo().

For example, consider this exported Rust type and constructor annotation:

#[wasm_bindgen]
pub struct Foo {
    contents: u32,
}

#[wasm_bindgen]
impl Foo {
    #[wasm_bindgen(constructor)]
    pub fn new() -> Foo {
        Foo { contents: 0 }
    }

    pub fn get_contents(&self) -> u32 {
        self.contents
    }
}

This can be used in JavaScript as:

import { Foo } from './my_module';

const f = new Foo();
console.log(f.get_contents());
```

La définition de la structure et son implémentation  doivent etre préfixés par la macro `#[wasm_bindgen]`, on rajoute l'attribut constructeur `#[wasm_bindgen(constructor)]` afin de pouvoir utiliser `new` depuis JS. Ainsi, la structure devient importable, ses variables et ses méthodes disponibles.

*Ne pas oublier `pub` pour le constructeur.*

## Méthodes et champs

### Forme générale

Afin de pouvoir partager les méthodes de l'implémentation ainsi que les champs, **il est nécessaire de les annoter comme public `pub`.**

```
#[wasm_bindgen]
pub struct Context {
    content: String,
    pub value: usize, // Je peux acceder et modifier cette variable depuis JS
} 
#[wasm_bindgen]
impl Context {
    #[wasm_bindgen(constructor)]
    pub fn new() -> Self {
        Self { content: String::new(), value: 42 }
    }
    pub fn dump(&self) -> String {
        format!("content = {}, value = {}", self.content, self.value)
    }
}
```

### Getter et Setter

Pour qu'une variable puisse etre partagée, il est nécessaire qu'elle implémente le trait `Copy`, ce n'est pas le cas pour String. Ainsi, si l'on veut pouvoir lire ou modifier ou lire un type String, il faudra mettre en place un getter et un setter - The `wasm-bindgen` Guide chapitre **2.18.2.9** :

**RUST side :**
```
#[wasm_bindgen]
pub struct Context {
    content: String,
    pub value: usize,
}                                                                                                                                                                            
                                                                                                                                                                             
#[wasm_bindgen]
impl Context {
    #[wasm_bindgen(constructor)]
    pub fn new() -> Self {
        Self { content: String::new(), value: 42 }
    }                                                                                                                                                                        
    #[wasm_bindgen(getter)]
    pub fn content(&self) -> String {
        self.content.clone()
    }                                                                                                                                                                        
    #[wasm_bindgen(setter)]
    pub fn set_content(&mut self, content: &str) {
        self.content = content.to_string();
    }
}
```
**JAVASCRIPT side :**
```
// Some importation code ...
let ctx = new Context();
ctx.content = "Banane";         // use setter
console.log(ctx.content);       // use getter
```

### L'attribut ReadOnly

- L'attribut `#[wasm_bindgen(readonly)]` peut etre intéressant pour empêcher les modifications d'une valeur depuis l'extérieur, sans vouloir non plus avoir une variable privée :

The `wasm-bindgen` Guide chapitre **2.18.2.3** :

```
readonly

When attached to a pub struct field this indicates that it's read-only from JavaScript, and a setter will not be generated and exported to JavaScript.

#[wasm_bindgen]
pub fn make_foo() -> Foo {
    Foo {
        first: 10,
        second: 20,
    }
}

#[wasm_bindgen]
pub struct Foo {
    pub first: u32,

    #[wasm_bindgen(readonly)]
    pub second: u32,
}

Here the first field will be both readable and writable from JS, but the second field will be a readonly field in JS where the setter isn't implemented and attempting to set it will throw an exception.

import { make_foo } from "./my_module";

const foo = make_foo();

// Can both get and set `first`.
foo.first = 99;
console.log(foo.first);

// Can only get `second`.
console.log(foo.second);
```

## L'attribut inspectable

> Un dernier attribut intéressant est inspectable `#[wasm_bindgen(inspectable)]`, il permet d'appeler toString() et toJSON() sur les structures Rust. Seuls les champs dits publics seront visibles. Notez que les implémentations peuvent etre redéfinies manuellement. The `wasm-bindgen` Guide chapitre **2.18.2.10**.

## Exercice

**REPRENEZ LE CODE DE LA PARTIE PRÉCÉDENTE ET AU LIEU D'APPELER DES FONCTIONS RUST DEPUIS JS, CRÉEZ UNE STRUCTURE EN RUST DE FAÇON À POUVOIR STOCKER LES CARACTÈRES ASSOCIES AUX TOUCHES APPUYÉES. SI L'UTILISATEUR APPUIE SUR LA TOUCHE ENTRÉE, LA CHAINE DOIT S'AFFICHER DANS LE TERMINAL.**